In [ ]:
#trying DBScan
#define intensity threshold. Points which have a lower intensity will be excluded from the clustering.
int_threshold=0

#apply the threshold on the vesicle and synaptic marker datasets
vesicles_thresh = vesicles[vesicles[:,3]>int_threshold,0:3]
synapse_marker_thresh = synapse_marker[synapse_marker[:,3]>int_threshold, 0:3]

#compute the fraction of points including in the clustering.
frct_points_included = vesicles_thresh.shape[0]/vesicles.shape[0]*100
print('% points above threshold:', frct_points_included)

# Normalize the data to have zero mean and unit variance
data_norm = (vesicles_thresh - np.mean(vesicles_thresh, axis=0)) / np.std(vesicles_thresh, axis=0)

# Apply DBSCAN to the normalized data
dbscan = DBSCAN(eps=800, min_samples=150)
labels = dbscan.fit_predict(vesicles_thresh)

#compute the number of clusters rendered by DBSCAN
unique_labels = np.unique(labels[labels >= 0])
n_clusters = len(unique_labels)
print('number of clusters:', n_clusters)

plt.scatter(vesicles[:,0], vesicles[:,1], s=0.1, alpha=0.5)
for label in unique_labels:
    mask = labels == label
    if np.any(mask):
        plt.scatter(vesicles_thresh[mask, 0], vesicles_thresh[mask, 1], s=20)

plt.title("DBSCAN Clusters")
plt.legend()

# Show the plot
plt.show()
plt.tight_layout()

In [ ]:
min_distances = {}
for i in keywords:
    for j in range(0, len(file_dic[i])):
            min_dist = fcts.calc_distance_squared_two(file_dic[i][j], file_dic[i][j])
            if i in min_distances:
                min_distances[i] = np.append(min_distances[i], min_dist)
            else:
                min_distances[i] = np.array([min_dist])

In [ ]:
#point pattern analysis in 2D

import pointpats
coordinates=pd.DataFrame(vesicle_clusters['210404 SPON647_PSD680 10DIV_CellZone1'][1][:,0:2], columns = ['x', 'y'])

g_test = pointpats.distance_statistics.g_test(coordinates, support=40, keep_simulations=True)

f, ax = plt.subplots(
    1, 2, figsize=(9, 3), gridspec_kw=dict(width_ratios=(6, 3))
)
# plot all the simulations with very fine lines
ax[0].plot(
    g_test.support, g_test.simulations.T, color="k", alpha=0.01
)
# and show the average of simulations
ax[0].plot(
    g_test.support,
    np.median(g_test.simulations, axis=0),
    color="cyan",
    label="median simulation",
)


# and the observed pattern's G function
ax[0].plot(
    g_test.support, g_test.statistic, label="observed", color="red"
)

# clean up labels and axes
ax[0].set_xlabel("distance")
ax[0].set_ylabel("% of nearest neighbor\ndistances shorter")
ax[0].legend()
#ax[0].set_xlim(0, 2000)
ax[0].set_title(r"Ripley's $G(d)$ function")

# plot the pattern itself on the next frame
ax[1].scatter(*coordinates)

# and clean up labels and axes there, too
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
ax[1].set_title("Pattern")
f.tight_layout()
plt.show()

In [ ]:
#point density analysis in 2d

# Set up figure and axis
f, ax = plt.subplots(1, figsize=(6, 6))
# Generate and add KDE with a shading of 50 gradients
# coloured contours, 75% of transparency,
# and the reverse viridis colormap
seaborn.kdeplot(coordinates, x='x',y='y',
    n_levels=50,
    shade=True,
    alpha=0.55,
    cmap="viridis_r",
)

# Remove axes
ax.set_axis_off()

In [ ]:
#loading a single file as example using outdated fct.

ex_zone = ['/Volumes/STORM_Nathalie/STORM DeMixing/210414 DEP647_PSD680 8DIV/CellZone4/Demix/CoordTable_SAFE360_MULTIPLEXING_demixed_w1_UncertaintyFiltered.csv',
  '/Volumes/STORM_Nathalie/STORM DeMixing/210414 DEP647_PSD680 8DIV/CellZone4/Demix/CoordTable_SAFE360_MULTIPLEXING_demixed_w2_UncertaintyFiltered.csv']

ex_zone = list_of_files[0]

vesicles = pd.read_csv(ex_zone[0])[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)
synapse_marker = pd.read_csv(ex_zone[1])[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)

data=vesicles
image_size = (730,730,16)
kernel_size = (50,50,1)
sigma = 12

wide_field_image = fcts.get_wide_field(vesicles, image_size, kernel_size, sigma)

coords = np.array(coords)

plt.imshow(image[:,:,0])
plt.scatter(coords[:,1], coords[:,0], c='red')

In [ ]:
#minimum distance for clusters of vesicles to PSD95 areas - but done as point location instead of point density

image_size = (730,730,16)
kernel_size = (50,50,1)
sigma = 15

max_threshold_ves = 12
max_threshold_mark = 4
max_area = (21,21,21)

min_dist_marker = {}
min_dist_vesicle = {}
SNR_dict = {}
for file_name in list_of_files:
    
    if access == 'drive':
        new_file_name = f"{(file_name[0]).split('/')[4]}_{(file_name[0]).split('/')[5]}"
    
    elif access == 'computer':
        new_file_name = f"{(file_name[0]).split('/')[-1][0:-3]}"
    
    print(new_file_name)
    
    file_info = files_infos[new_file_name]
    
    
    if file_info[0] == 0:
        pass
    
    else:
        vesicles = pd.read_csv(file_name[file_info[1]])[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)
        synapse_marker = pd.read_csv(file_name[(file_info[1]-1)**2])[['x [nm]', 'y [nm]', 'z [nm]']].to_numpy(dtype=np.float64)
        
        #plot the locations of both the vesicles and the synaptic marker using a large point spread function.
        wide_field_vesicles = fcts.get_wide_field(vesicles, image_size, kernel_size, sigma)
        wide_field_marker = fcts.get_wide_field(synapse_marker, image_size, kernel_size, sigma)
        
        #plot the locations of the vesicles using the same image size as above to a single pixel size
        image_vesicles = fcts.get_wide_field(vesicles, image_size, (1,1,1), sigma)
        
        #calculate the intensity threshold for the large PSF images, depending on an arbitrary intensity threshold, dependent on the mean and std of each image.
        ves_thresh = wide_field_vesicles.mean() + wide_field_vesicles.std() * max_threshold_ves
        marker_thresh = wide_field_marker.mean() + wide_field_marker.std() * max_threshold_mark
        
        #create a mask of the large PSF images where for the pixels above the threshold
        mask_vesicles = (wide_field_vesicles > ves_thresh) * 1

        plt.imshow(mask_vesicles[:,:,0])
        plt.show()
        mask_marker = (wide_field_marker > marker_thresh) * 1

        #create an array for the vesicles which are located within the mask
        masked_vesicles = np.where(mask_vesicles, image_vesicles, 0)
        vesicle_pos_tup = np.where(masked_vesicles > 0)
        vesicle_pos = np.array([vesicle_pos_tup[0],vesicle_pos_tup[1],vesicle_pos_tup[2]]).T
        vesicle_pos = vesicle_pos/image_size * 49660
        
        nb_vesicles_tot = vesicles.shape[0]
        nb_vesicles_selected = vesicle_pos.shape[0]
        snr = nb_vesicles_selected/nb_vesicles_tot
        
        #calculate the distance between vesicles within the masked images and all synapse markers
        distance_to_marker = fcts.calc_distance_squared_two(vesicle_pos, synapse_marker)
        distance_to_vesicles = fcts.calc_distance_squared_two(vesicle_pos, vesicle_pos)
        print(distance_to_vesicles)
        min_dist_marker[new_file_name] = distance_to_marker
        min_dist_vesicle[new_file_name] = distance_to_vesicles
        SNR_dict[new_file_name] = snr
        
        
print('done')

In [ ]:
import numpy as np

def merge_points(points, diameter=40):
    """
    Merges points that are within `diameter` distance of each other and returns a new array with one point per vesicle
    """
    merged_points = []
    while len(points) > 0:
        p = points[0]
        points = np.delete(points, 0, axis=0)
        nearby_points = np.linalg.norm(points - p, axis=1) < diameter
        while np.sum(nearby_points) > 0:
            p = np.mean(np.concatenate([points[nearby_points], [p]]), axis=0)
            points = np.delete(points, np.where(nearby_points)[0], axis=0)
            nearby_points = np.linalg.norm(points - p, axis=1) < diameter
        merged_points.append(p)
    return np.array(merged_points)


# Example dataset of 100 points with x,y,z coordinates
points = np.random.rand(100, 2)

# Merge points within 40nm of each other
merged_points = merge_points(points, diameter=0.1)

# Check the number of original points vs merged points
print(f"Original points: {points.shape[0]}, Merged points: {merged_points.shape[0]}")

plt.scatter(points[:,0], points[:,1], alpha=0.5)
plt.scatter(merged_points[:,0], merged_points[:,1], alpha=0.5)
plt.show()

In [ ]:
#permutation test

x = vesicle_type_nd_DIV['DEP647_PSD680 8DIV']
y = vesicle_type_nd_DIV['SPON647_PSD680 8DIV']


def statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

from scipy.stats import permutation_test
# because our statistic is vectorized, we pass `vectorized=True`
# `n_resamples=np.inf` indicates that an exact test is to be performed
res = permutation_test((x, y), statistic, vectorized=True,
                       n_resamples=9999, alternative='less')
print(res.statistic)
print(res.pvalue)


In [ ]:
#unused functions


@njit(fastmath=True,parallel=True)
def calc_distance_squared_two(vec_1,vec_2):

    res=np.empty((vec_1.shape[0]),dtype=vec_1.dtype)
    for i in prange(vec_1.shape[0]):
        dist= np.sqrt((vec_1[i,0]-vec_2[:,0])**2+(vec_1[i,1]-vec_2[:,1])**2+(vec_1[i,2]-vec_2[:,2])**2)
        res[i] = np.min(dist[np.nonzero(dist)])
    return res

    
def get_wide_field(data, image_size, kernel_size, sigma):
    
    #prep data for image
    data[:,2] =+ 485
    data = data/49660*image_size
    
    #initialise background with a buffer
    buffered_image_dim = tuple(map(lambda i, j: i + j, image_size, kernel_size))
    buffered_image = np.zeros(buffered_image_dim)

    #initialise the gaussian distribution with size being the size of the kernel and sigma defining the standard deviation of the point spread function
    kernel = np.fromfunction(lambda x, y, z : (1/(2*np.pi*sigma**2)) * np.exp((-1*((x-(kernel_size[0]-1)/2)**2+(y-(kernel_size[1]-1)/2)**2+(z-(kernel_size[2]-1)/2)**2)/(2*sigma**2))), kernel_size)
    kernel = kernel / np.max(kernel)

    #get the pixel location of the data
    point_coord = np.round(data).astype(int)
    #add gaussian point spread function at each point location
    for y, x, z in point_coord:
        buffered_image[x:x+kernel.shape[0], y:y+kernel.shape[1], z:z+kernel.shape[2]] += kernel
    
    #remove buffer from image
    image = buffered_image[kernel.shape[0]//2:-kernel.shape[0]//2, kernel.shape[1]//2:-kernel.shape[1]//2, kernel.shape[2]//2:-kernel.shape[2]//2]
    
    return image


def get_local_max_3d(img, kernelsize, std_threshold):
    # Get coordinates of local maxima 

    img2 = ndimage.maximum_filter(img, size=kernelsize)

    # Threshold the image to find locations of interest
    img_thresh = img2.mean() + img2.std() * std_threshold

    # Since we're looking for maxima find areas greater than img_thresh
    labels, num_labels = ndimage.label(img2 > img_thresh)

    # Get the positions of the maxima
    coords = ndimage.center_of_mass(img, labels=labels, index=np.arange(1, num_labels + 1))
    coords = np.array(coords)
    # Get the maximum value in the labels    

    return coords

In [ ]:
def normalize_array(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

# apply the normalization function to the column of arrays
storm_data['nn_680_normalized'] = storm_data['nearest_neighbor_680'].apply(normalize_array)


# concatenate the normalized arrays into a single numpy array
dep_array = np.concatenate(storm_data[storm_data['647nm']=='DEP647']['nearest_neighbor_680'].to_numpy())
spon_array = np.concatenate(storm_data[storm_data['647nm']=='SPON647']['nearest_neighbor_680'].to_numpy())

n, x, _ = plt.hist(dep_array, bins=200, 
                   histtype=u'step', density=True)  

density_dep = sc.gaussian_kde(dep_array)
density_spon = sc.gaussian_kde(spon_array)


plt.plot(x, density_dep(x), alpha=0.5)
plt.plot(x, density_spon(x), alpha=0.5)

plt.show()


dep_array = np.sort(dep_array)
spon_array = np.sort(spon_array)

dep_array_sorted = dep_array[:-1].reshape((8, dep_array[:-1].shape[0]//8))
dep_array_sorted_mean = np.mean(dep_array_sorted, axis=1)
spon_array_sorted = spon_array[:-1].reshape((8, spon_array[:-1].shape[0]//8))
spon_array_sorted_mean = np.mean(spon_array_sorted, axis=1)

plt.scatter(x=[np.zeros(8), np.ones(8)], y = [spon_array_sorted_mean, dep_array_sorted_mean])
plt.show()

In [ ]:
dep_array = storm_data[storm_data['647nm']=='DEP647']['nearest_neighbor_680'].to_numpy()
spon_array = storm_data[storm_data['647nm']=='SPON647']['nearest_neighbor_680'].to_numpy()
    
quarter_len = storm_data['nearest_neighbor_680'].apply(lambda x: len(x) // 4)
quarter_len = quarter_len.to_numpy()
# separate the array into quarters and calculate the mean of each quarter
quarter_means = pd.DataFrame()
for i in range(4):
    quarter_arr = storm_data['nearest_neighbor_680'].apply(lambda x: x[i*quarter_len:(i+1)*quarter_len])
    quarter_mean = quarter_arr.apply(lambda x: np.mean(x))
    quarter_means[f'quarter_{i+1}_mean'] = quarter_mean

print(quarter_means)

In [ ]:
# concatenate the normalized arrays into a single numpy array
dep_array_norm = np.concatenate(storm_data[storm_data['647nm']=='DEP647']['nn_680_normalized'].to_numpy())
spon_array_norm = np.concatenate(storm_data[storm_data['647nm']=='SPON647']['nn_680_normalized'].to_numpy())

n, x, _ = plt.hist(dep_array, bins=50, 
                   histtype=u'step', density=True)  


density_dep = sc.gaussian_kde(dep_array_norm)
density_spon = sc.gaussian_kde(dep_array_norm)


plt.plot(x, density_dep(x))
plt.plot(x, density_spon(x))

plt.show()

In [ ]:
#compute the n shortest distances from one vesicles to its n neighbors for each vesicle

n = 50

all_points = storm_data['points'].to_numpy()

NNDs = np.array([])
for points in all_points:
    tree = KDTree(points)
    nearest_dist, nearest_ind = tree.query(points, k=n+1) 
    NNDs = np.concatenate((NNDs, nearest_dist), axis=None)

NNDs = NNDs.reshape((int(NNDs.shape[0]/(n+1)), n+1))
NNDs.shape
NNDs_df  = pd.DataFrame(NNDs, columns = np.array(range(0,n+1)))


In [ ]:

# Create a scatter plot
#plt.scatter(NNDs_df.columns.values, NNDs_df.mean(axis=0))
plt.plot(np.unique(NNDs_df.columns.values), np.poly1d(np.polyfit(NNDs_df.columns.values, NNDs_df.mean(axis=0), 1))(np.unique(NNDs_df.columns.values)))

# Set the  -axis label
plt.xlabel('Nearest Neighbor Index')

# Set the y-axis label
plt.ylabel('Mean Nearest Neighbor Distance')

# Set the plot title
plt.title('Nearest Neighbor Distance')

# Show the plot
plt.show()

In [ ]:
#trying to look at the intervesicular distance for the nth order of nearest neighbour. very computationally heavy, nothing very interesting

n=32
avg_nnd = pd.DataFrame()
for condition in ['SPON647', 'DEP647']:
    nnd_array = np.array(storm_data[storm_data['647nm'] == condition]['15_NNs_647'].to_list(), dtype=object)
    nnd_array = np.concatenate((nnd_array), axis=None)
    nnd_array = nnd_array.reshape((int(nnd_array.shape[0]/(n+1)), n+1))
    
    avg_nnd[condition] = np.mean(nnd_array, axis=0)
    print(avg_nnd[condition].shape)
    
# Step 2: Plot average nearest neighbor distances for each condition
fig, ax = plt.subplots()
ax.plot(avg_nnd)
ax.set_xlabel('Rank')
ax.set_ylabel('Average nearest neighbor distance')
plt.show()

plt.plot(avg_nnd, alpha=0.5)

In [ ]:
def project_3d_to_2d(arr):
    # Get the maximum value of the array along the z-axis
    # Get the sum of the maximum values along the x and y axes
    projection = np.sum(arr, axis=2)
    
    return projection

for file_name in list(synapses.keys()):
    print(file_name)
    im = project_3d_to_2d(synapses[file_name])
    plt.imshow(im)
    plt.show()


In [ ]:
    
#VISUALISE

for i in list(vesicle_clusters.keys()):
    for j in list(vesicle_clusters[i].keys()):
        print(i)
        print(j)
        print(vesicle_clusters[i][j].shape[0])
        
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')

        ax.scatter(vesicle_clusters[i][j][:,0], vesicle_clusters[i][j][:,2], vesicle_clusters[i][j][:,1], c = 'r', marker='o', alpha=0.2)
    
        plt.show()

In [ ]:
density_est = {}
mean_dens = {}
for i in keywords:
    print(i)
    for j in range(0, len(data_sorted[i])):
            data = data_sorted[i][j].T

            #determine bounding box for the density estimation of points in the synapse
            xmin = data[0].min()
            xmax = data[0].max()
            ymin = data[1].min()
            ymax = data[1].max()
            zmin = data[2].min()
            zmax = data[2].max()
            
            #get kde based on data
            kde = sc.gaussian_kde(data, 10)
            
            #define the kernel size in all dimensions
            xstep = int((xmin-xmax)/40)
            ystep = int((ymin-ymax)/40)
            zstep = int((zmin-zmax)/40)
            
            #projec the kde onto a grid of points within the bounding box given a timestep
            X, Y, Z = np.mgrid[xmin:xmax:(xstep*1j), ymin:ymax:(ystep*1j), zmin:zmax:(zstep*1j)]
            positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])
            kde_est = np.reshape(kde(positions).T, X.shape)
            mean = np.mean(kde_est)
            var = np.std(kde_est)
            if i in density_est:
                density_est[i] = np.append(density_est[i], kde_est)
            else:
                density_est[i] = kde_est
                
            if i in mean_dens:
                mean_dens[i] = np.append(mean_dens[i], [mean, var])
            else:
                mean_dens[i] = [mean, var]


In [ ]:
threshold=1e-10
#list_of_thresholds = list(range(0, threshold, 10))

file_dic = min_coloc_dist

for key in list(file_dic.keys()):
    #data = file_dic[key][(file_dic[key]<threshold)]
    data = file_dic
    #data = data.flatten()
    plt.title(key)
    plt.hist(data, bins = 30, label= list(file_dic.keys()), alpha=0.5, density=True)
    plt.xlabel('density', fontsize = 18)
    plt.ylabel('cumulated area', fontsize = 18)
    plt.xticks(fontsize = 14)
    plt.yticks(fontsize = 14)
plt.show()

plt.boxplot(list(mean_dens.values()), showmeans=True, labels=list(file_dic.keys()))
plt.xticks(fontsize=6)
plt.title(f'average density profile')

for i in range(0, len(list(file_dic.keys()))):
    stats = sc.describe(file_dic[list(file_dic.keys())[i]])
    print(list(file_dic.keys())[i], stats)
    

In [ ]:
#compute the nearest neighbor distance and ratio

def getArea(points):
    """returns a list containing the bottom left and the top right 
    points in the sequence
    Here, we use min and max four times over the collection of points
    """
    xmin = points[:,0].min()
    xmax = points[:,0].max()
    ymin = points[:,1].min()
    ymax = points[:,1].max()
    zmin = points[:,2].min()
    zmax = points[:,2].max()

    return (xmax - xmin) * (ymax - ymin) * (zmax - zmin)

NND = {}
NNR = {}
for i in keywords:
    for array in data_sorted[i]:
        tree = KDTree(array)
        nearest_dist, nearest_ind = tree.query(array, k=2) 
        nearest_dist = nearest_dist[:,-1]
        
        mean_NND = sum(nearest_dist)/len(array)

        area = getArea(array)
        exp_mean_NND =  0.5/np.sqrt(len(array)/area)     
        NNR_t = mean_NND/exp_mean_NND
        
        if i in NND:
            NND[i] = np.concatenate((NND[i], nearest_dist))
        else:
            NND[i] = nearest_dist.flatten()
            
        if i in NNR:
            NNR[i] = np.concatenate((NNR[i], [NNR_t]))
        else:
            NNR[i] = NNR_t.flatten()


In [ ]:
for i in range(0, len(list(NND.keys()))):
    plt.title(list(NND.keys())[i])
    plt.hist(NND[list(NND.keys())[i]], bins = 20)

plt.plot(list(NNR.values()), labels=list(file_dic.keys()))


In [ ]:
from sklearn.cluster import KMeans
random.seed(0)
simulation, sim_params = fcts.simulation_batch(5, [17,17,50], [1000,1000,20])

peak_coords, sim_clusters, sim_vesicles = fcts.get_clustered_points(simulation[0], params)

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

peak_coords_plot = peak_coords*params['sf']

ax.scatter(simulation[0][:, 0], simulation[0][:, 1], simulation[0][:, 2], marker='o', alpha=0.5)
ax.scatter(peak_coords_plot[:, 0], peak_coords_plot[:, 1], peak_coords_plot[:, 2], c='black')

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Display the plot
plt.show()
print(np.max(sim_clusters))


In [ ]:
#lets try pca 
storm_data['points_norm'] = storm_data.apply(lambda x: x['points']- x['point_center'], axis=1)

from sklearn.decomposition import PCA
storm_data['PCAs'] = storm_data['points_norm'].apply(lambda x:PCA(n_components=1).fit_transform(x))
storm_data['PCAs'] = storm_data['PCAs'].apply(lambda x:x.flatten().tolist())
storm_data['PCAs variation'] = storm_data['points_norm'].apply(lambda x:PCA(n_components=1).fit(x).explained_variance_ratio_)
storm_data['PCAs variation'] = storm_data['PCAs variation'].apply(lambda x:x[0])

In [ ]:
i = 1001
arr = np.sort(storm_data['PCAs'][i][:,0])
plt.scatter(np.linspace(0,10,storm_data['PCAs'][i].shape[1]), arr, s=1)
print(storm_data['markertype and DIV'][i])

In [ ]:
from scipy.stats import norm
dict_pcapoints = {'DEP647':[], 'SPON647':[]}

for i, group in enumerate(storm_data.groupby('647nm')):
    for j, row in group[1].iterrows():
        for item in row['PCAs']:
            dict_pcapoints[group[0]].append(item)

#plt.hist(dep_10_r, density=True, alpha=0.5)
#plt.hist(dep_8_r, density=True, alpha=0.5)
plt.hist(dict_pcapoints['DEP647'], bins=100, density=True, alpha=0.5)
plt.hist(dict_pcapoints['SPON647'], bins=100, density=True, alpha=0.5)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
s_dep = np.std(dict_pcapoints['DEP647'])
m_dep = np.mean(dict_pcapoints['DEP647'])

s_spon = np.std(dict_pcapoints['SPON647'])
m_spon = np.mean(dict_pcapoints['SPON647'])

p_spon = norm.pdf(x, m_spon, s_spon)
p_dep = norm.pdf(x, m_dep, s_dep)

plt.plot(x, p_spon, linewidth=2, color='red')
plt.plot(x, p_dep, 'k', linewidth=2, color='blue')



plt.show()


In [ ]:
def gaussian_peak_est(data, step):
    #determine bounding box for the density estimation of points in the synapse
    data = data.T
    xmin = data[0].min()
    xmax = data[0].max()
    ymin = data[1].min()
    ymax = data[1].max()
    zmin = data[2].min()
    zmax = data[2].max()

    #get kde based on data
    kde = sc.gaussian_kde(data)

    #define the kernel size in all dimensions
    xstep = int((xmin-xmax)/step)
    ystep = int((ymin-ymax)/step)
    zstep = int((zmin-zmax)/step)

    #projec the kde onto a grid of points within the bounding box given a timestep
    X, Y, Z = np.mgrid[xmin:xmax:(xstep*1j), ymin:ymax:(ystep*1j), zmin:zmax:(zstep*1j)]
    positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])
    kde_est = np.reshape(kde(positions).T, X.shape)
    
    return kde_est

storm_data['gaussian_project'] = storm_data.apply(lambda row: gaussian_peak_est(row['points'],30), axis=1)

def get_cluster_peaks(wide_field_vesicles, params):
    
    max_threshold_ves = params['max_threshold_ves']
    min_peak_distance = params['min_peak_distance']
    
    ves_thresh = wide_field_vesicles.mean() + wide_field_vesicles.std() * max_threshold_ves
    mask_vesicles = (wide_field_vesicles > ves_thresh) * 1
    peak_coords = peak_local_max(wide_field_vesicles, min_distance=min_peak_distance)
    print(len(peak_coords))
    #if len(peak_coords)!= 0:
    #    peak_coords = np.array(fcts.filter_peaks(peak_coords, min_peak_distance))
    return peak_coords

params['min_peak_distance'] = 15

storm_data['cluster_peaks'] = storm_data.apply(lambda row: fcts.get_local_max_3d(row['gaussian_project'],(1,1,1),1), axis=1)
storm_data['number_of_peaks'] = storm_data['cluster_peaks'].apply(len)

In [ ]:
plt.plot(storm_data['PCAs'][1])

data = storm_data['gaussian_project'][733]
peaks = storm_data['cluster_peaks'][733]
plt.figure(1)
plt.clf()
print(peaks)
plt.imshow(data[:,:,3])
plt.plot(peaks[:, 1],peaks[:, 0],'rx')

In [ ]:
x = '647nm'
y = 'number_of_peaks'

ttest, pval = sc.ttest_ind(storm_data[storm_data['647nm']=='DEP647']['number_of_peaks'], 
                           storm_data[storm_data['647nm']=='SPON647']['number_of_peaks'])

# Set up plot
fig, axes = plt.subplots(1, 1, figsize=(6, 6))

# Volume plot
sns.violinplot(data=storm_data.sort_values(by=[x]), x=x, y=y, 
             linewidth=0, palette='colorblind')
sns.stripplot(data=storm_data.sort_values(by=[x]), x=x, y=y, jitter=True, color='black', alpha=0.7, size=2)

plt.xlim([-0.5, len(storm_data[x].unique())-0.5])
plt.xlabel('labelled vesicle pool')
plt.xticks(ticks = [0,1],labels = ['depolarised', 'spontaneous'])

for i, group in enumerate(storm_data.groupby(x)):
    mean_val = group[1][y].mean()
    plt.plot([i-0.2, i+0.2], [mean_val, mean_val], 'k-', lw=2)

plt.annotate(f'p-value = {pval:6f}',
            xy=(1.4, 880),
            horizontalalignment='right', verticalalignment='top')    
    
plt.show()